In [2]:
from PIL import Image
import os
import numpy as np
import random

maxWidth = 1920
maxHeight = 1080

def generateBgNoise(width,height):
    im = Image.new('RGB',(width, height))
    pixels = im.load()
    for x in range(im.size[0]):
        for y in range(im.size[1]):
            pixels[x, y] = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
            
    return im

def generateScales(width,height):
    scaleRatios = [0.15,0.4,0.65,0.9]
    output = np.empty((len(scaleRatios),2))
    
    resRatio = width/height
    ratioCondition = width/maxWidth > height/maxHeight
        
    for r in range(len(scaleRatios)):
        ratio = scaleRatios[r]
        
        if ratioCondition:
            nWidth = ratio*maxWidth
            nHeight = nWidth*(1/resRatio)
            output[r,:] = [int(nWidth),int(nHeight)]
            
        else:
            nHeight = ratio*maxHeight
            nWidth = nHeight*resRatio
            output[r,:] = [int(nWidth),int(nHeight)]
            
    return output

def generateCoords(width,height):
    widthOff = maxWidth - width
    heightOff = maxHeight - height
    
    if heightOff > 0.6*maxHeight:
        heightOff = 0.6*maxHeight
    
    output = np.empty((4,2))
    
    for i in range(4):
        output[i,0] = random.randint(0,widthOff)
        output[i,1] = random.randint(0,heightOff)
        
    return output

## Synthetic data generator

Change the file paths depending on which images you want to use in your data generation and where you want to save the output data.

In [ ]:
index = 0

for i in range(1,23):
    doorImage = Image.open(os.getcwd() + "/syn-nonDoor-templates/" + str(i) + ".png")
    width, height = doorImage.size
    
    scales = generateScales(width, height)
    
    for s in range(4):
        rescaledDoor = doorImage.resize((int(scales[s,0]), int(scales[s,1])))
        coords = generateCoords(int(scales[s,0]), int(scales[s,1]))
        
        for c in range(4):
            print(index)
            
            im = generateBgNoise(maxWidth,maxHeight)
            im.paste(rescaledDoor,(int(coords[c,0]),int(coords[c,1])),rescaledDoor)
            
            im = im.save(os.getcwd() + "/syn-nonDoor-autogen/" + str(index) + ".png")
            index += 1
            
print("done")

## Data compiler and randomiser

In [26]:
order = np.arange(704)
random.shuffle(order)

train = np.empty((564))
trainBalance = [0,0]
test = np.empty((140))
testBalance = [0,0]

for index in range(704):
    element = order[index]
    
    isDoor = 0
    if element < 352:
        isDoor = 1
        
    if trainBalance[isDoor] < 282:
        train.put(trainBalance[0] + trainBalance[1], element)
        trainBalance[isDoor] += 1
    else:
        test.put(testBalance[0] + testBalance[1], element)
        testBalance[isDoor] += 1
        

trainY = np.empty((564))
for s in range(len(train)):
    sample = train[s]
    
    fileName = "/syn-door-autogen/" + str(int(sample))
    if sample > 351:
        fileName = "/syn-nonDoor-autogen/" + str(int(sample) - 352)
        trainY.put(s, 0)
    else:
        trainY.put(s, 1)
        
    doorImage = Image.open(os.getcwd() + fileName + ".png")
    
    doorImage = doorImage.save(os.getcwd() + "/dataset/training/" + str(s) + ".png")

np.save(os.getcwd() + "/dataset/training/trainLabels.npy", trainY)

testY = np.empty((140))
for t in range(len(test)):
    sample = test[t]
    
    fileName = "/syn-door-autogen/" + str(int(sample))
    if sample > 351:
        fileName = "/syn-nonDoor-autogen/" + str(int(sample) - 352)
        testY.put(t, 0)
    else:
        testY.put(t, 1)
        
    doorImage = Image.open(os.getcwd() + fileName + ".png")
    
    doorImage = doorImage.save(os.getcwd() + "/dataset/testing/" + str(t) + ".png")
    
np.save(os.getcwd() + "/dataset/testing/testLabels.npy", testY)

140
